# ReAct: Reasoning + Acting

Building on:
- **001-003**: Basic Branch usage and function calling
- **004**: Conversation patterns and context

**This notebook**: Introduction to ReAct - getting AI to think step-by-step and use tools strategically.

In [1]:
from lionagi import Branch, iModel
import json
from IPython.display import Markdown

## What is ReAct?

**ReAct** = **Reasoning** + **Acting**

Instead of just giving an answer, the AI:
1. **Thinks** about the problem (reasoning)
2. **Uses tools** when needed (acting)
3. **Repeats** until the problem is solved

This creates more reliable, step-by-step problem solving.

## Simple Example: Math Problem Solver

Let's start with a basic calculator that shows its reasoning:

In [2]:
# Simple calculator functions
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b


def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b


def divide(a: float, b: float) -> float:
    """Divide two numbers."""
    if b == 0:
        return "Error: Cannot divide by zero"
    return a / b


# Create a ReAct-enabled branch
calculator = Branch(
    system="You solve math problems step by step. Think about what you need to calculate, then use tools.",
    tools=[add, multiply, divide],
    chat_model=iModel(model="openai/gpt-4.1-mini"),
)

print("Calculator ready with ReAct capabilities!")

Calculator ready with ReAct capabilities!


## Using ReAct for Problem Solving

In [3]:
# Problem: Calculate the total cost of 3 items at $15.50 each, plus 8% tax
problem = (
    "I bought 3 items at $15.50 each. What's the total cost including 8% tax?"
)

# Use ReAct to solve step-by-step
solution = await calculator.ReAct(
    instruct={"instruction": problem},
    max_extensions=5,  # Limit the number of reasoning steps
    verbose=True,  # Show the reasoning process
)

print("\n🎯 Final Answer:")
print(solution)

### ReAct Round No.1 Analysis:

```yaml
analysis: Step 1: Total cost of items before tax = 3 * 15.50 = $46.50. Step 2: Calculate 8% tax on $46.50 = 0.08 * 46.50. Step 3: Add tax to initial total to find total cost including tax.
planned_actions:
  - action_type: arithmetic
    description: Calculate 3 times 15.50 for total item cost.
  - action_type: arithmetic
    description: Calculate 8% tax on total item cost.
  - action_type: arithmetic
    description: Add tax to total item cost to get final amount.
extension_needed: True
milestone: Calculate total cost including 8% tax
action_strategy: sequential
reason:
  title: Calculate total cost with tax
  content: First, calculate the total price for 3 items at $15.50 each. Then calculate the 8% tax on that total. Finally, add the tax to the original total to get the total cost including tax.
  confidence_score: 1.0
```

### ReAct Round No.2 Analysis:

```yaml
analysis: We need to find the total cost before tax by multiplying the number of items (3) with the single item price ($15.50). This will be followed by calculating the tax and final total in the next steps.
planned_actions:
  - action_type: arithmetic
    description: Multiply 3 items by $15.50 to find total cost before tax.
extension_needed: True
milestone: Calculate total cost before tax
action_strategy: sequential
reason:
  title: Calculate total cost of items before tax
  content: First, calculate the total cost of 3 items priced at $15.50 each by multiplying 3 by 15.50.
  confidence_score: 1.0
action_required: True
action_requests:
  - function: multiply
    arguments:
      a: 3
      b: 15.5
action_responses:
  - function: multiply
    arguments:
      a: 3
      b: 15.5
    output: 46.5
```

### ReAct Round No.3 Analysis:

```yaml
analysis: Calculating 8% tax on the total cost of $46.50 will give the tax amount. We will then add this tax amount to the $46.50 to get the final amount payable.
planned_actions:
  - action_type: arithmetic
    description: Multiply total cost before tax (46.5) by 0.08 to calculate the 8% tax amount.
extension_needed: True
milestone: Calculate tax and prepare to add to total cost
action_strategy: sequential
reason:
  title: Calculate total tax and total cost including tax
  content: The total cost before tax is $46.50. Next, calculate 8% tax on $46.50, then add the tax amount to the total cost before tax to find the final cost including tax.
  confidence_score: 1.0
action_required: True
action_requests:
  - function: multiply
    arguments:
      a: 46.5
      b: 0.08
action_responses:
  - function: multiply
    arguments:
      a: 46.5
      b: 0.08
    output: 3.72
```

### ReAct Round No.4 Analysis:

```yaml
analysis: Adding the tax amount ($3.72) to the total cost before tax ($46.50) will give the final amount to be paid including tax.
planned_actions:
  - action_type: arithmetic
    description: Add $3.72 tax to $46.50 total cost before tax to get final total cost.
extension_needed: True
milestone: Calculate final total cost including tax
action_strategy: sequential
reason:
  title: Calculate total cost including tax
  content: The tax amount on $46.50 is $3.72. Now add this tax to the original total cost to get the final total including tax.
  confidence_score: 1.0
action_required: True
action_requests:
  - function: add
    arguments:
      a: 46.5
      b: 3.72
action_responses:
  - function: add
    arguments:
      a: 46.5
      b: 3.72
    output: 50.22
```

### ReAct Round No.5 Analysis:

```yaml
analysis: We first calculated the cost for 3 items at $15.50 each, which gave $46.50. Then calculated 8% tax on $46.50 as $3.72. Finally, we added the tax to the original cost to find the total $50.22.
planned_actions:
extension_needed: False
milestone: Final total cost calculated
action_strategy: sequential
reason:
  title: Final total amount calculation completed
  content: The total cost before tax is $46.50, the calculated 8% tax is $3.72, and the final total cost including tax is $50.22.
  confidence_score: 1.0
action_required: False
action_requests:
action_responses:
  - function: multiply
    arguments:
      a: 3
      b: 15.5
    output: 46.5
  - function: multiply
    arguments:
      a: 46.5
      b: 0.08
    output: 3.72
  - function: add
    arguments:
      a: 46.5
      b: 3.72
    output: 50.22
```

### ReAct Final Answer:

```yaml
answer: The total cost for 3 items at $15.50 each including 8% tax is $50.22.
```


🎯 Final Answer:
The total cost for 3 items at $15.50 each including 8% tax is $50.22.


## Understanding the ReAct Process

Let's look at what happened in the conversation:

In [4]:
# View the reasoning steps
conversation = calculator.to_df()
print(f"Total conversation steps: {len(conversation)}")
print("\nStep-by-step breakdown:")

for i, row in conversation.iterrows():
    if row["role"] == "user":
        print(f"\n👤 User: {str(row['content'])[:100]}...")
    elif row["role"] == "assistant":
        print(f"🤖 Assistant: {str(row['content'])[:100]}...")
    elif row["role"] == "action":
        print(f"🔧 Tool Used: {str(row['content'])[:100]}...")

Total conversation steps: 19

Step-by-step breakdown:

👤 User: {'instruction': "I bought 3 items at $15.50 each. What's the total cost including 8% tax?\n\nYou can...
🤖 Assistant: {'assistant_response': '```json\n{\n  "reason": {\n    "title": "Calculate total cost with tax",\n  ...

👤 User: {'instruction': 'You can perform multiple reason-action steps for accuracy. If you are not ready to ...
🤖 Assistant: {'assistant_response': '```json\n{\n  "reason": {\n    "title": "Calculate total cost of items befor...
🔧 Tool Used: {'function': 'multiply', 'arguments': {'a': 3, 'b': 15.5}, 'action_response_id': '1c37c1ff-dd04-4bc4...
🔧 Tool Used: {'function': 'multiply', 'arguments': {'a': 3, 'b': 15.5}, 'output': 46.5, 'action_request_id': 'fed...

👤 User: {'instruction': 'Another round is available. You may do multiple actions if needed. You have up to 4...
🤖 Assistant: {'assistant_response': '```json\n{\n  "reason": {\n    "title": "Calculate total tax and total cost ...
🔧 Tool Used: {'functio

## Practical Example: Research Assistant

Let's build a simple research assistant that can look up information:

In [5]:
# Simple "database" functions
company_data = {
    "apple": {"founded": 1976, "employees": 164000, "revenue": "$394.3B"},
    "google": {"founded": 1998, "employees": 182000, "revenue": "$307.4B"},
    "microsoft": {"founded": 1975, "employees": 238000, "revenue": "$211.9B"},
}


def lookup_company(name: str) -> str:
    """Look up information about a company."""
    name = name.lower()
    if name in company_data:
        data = company_data[name]
        return f"{name.title()}: Founded {data['founded']}, {data['employees']} employees, {data['revenue']} revenue"
    return f"No data found for {name}"


def calculate_age(founded_year: int) -> int:
    """Calculate how old a company is."""
    from datetime import datetime

    current_year = datetime.now().year
    return current_year - founded_year


# Research assistant with ReAct
researcher = Branch(
    system="You are a research assistant. Look up information and analyze it step by step.",
    tools=[lookup_company, calculate_age],
    chat_model=iModel(model="openai/gpt-4.1-mini"),
)

In [6]:
# Research question
question = "Which company is oldest: Apple, Google, or Microsoft? How much older is the oldest than the newest?"

# Let ReAct figure out the steps
research_result = await researcher.ReAct(
    instruct={"instruction": question}, max_extensions=10, verbose=True
)

print("\n📊 Research Result:")
print(research_result)

### ReAct Round No.1 Analysis:

```yaml
analysis: |
  Step 1: Identify the founding year of Apple.
  Step 2: Identify the founding year of Google.
  Step 3: Identify the founding year of Microsoft.
  Step 4: Compare the years to find the oldest and newest.
  Step 5: Calculate how many years older the oldest is compared to the newest.
planned_actions:
  - action_type: search_exa
    description: Search for the founding years of Apple, Google, and Microsoft to ensure accuracy.
extension_needed: True
milestone: Obtain founding years for all three companies.
action_strategy: concurrent
reason:
  title: Initial plan for determining company founding years
  content: To find which company is oldest among Apple, Google, and Microsoft, we need the founding years of each. Then calculate the age difference between the oldest and newest.
  confidence_score: 0.9
```

### ReAct Round No.2 Analysis:

```yaml
analysis: We need to lookup each company's founding year: Apple, Google, and Microsoft. Once we have all three founding years, we can identify the oldest and newest company and calculate the difference.
planned_actions:
  - action_type: lookup_company
    description: Lookup Apple's founding year.
  - action_type: lookup_company
    description: Lookup Google's founding year.
  - action_type: lookup_company
    description: Lookup Microsoft's founding year.
extension_needed: True
milestone: Obtain founding years of Apple, Google, Microsoft for comparison.
action_strategy: concurrent
reason:
  title: Look up founding years of Apple, Google, and Microsoft
  content: To accurately compare their ages, retrieval of each company's founding year is necessary. This ensures no memory-based errors and confirms current known founders/years.
  confidence_score: 0.95
action_required: True
action_requests:
  - function: lookup_company
    arguments:
      name: Apple
  - function: lookup_company
    arguments:
      name: Google
  - function: lookup_company
    arguments:
      name: Microsoft
action_responses:
  - function: lookup_company
    arguments:
      name: Apple
    output: Apple: Founded 1976, 164000 employees, $394.3B revenue
  - function: lookup_company
    arguments:
      name: Google
    output: Google: Founded 1998, 182000 employees, $307.4B revenue
  - function: lookup_company
    arguments:
      name: Microsoft
    output: Microsoft: Founded 1975, 238000 employees, $211.9B revenue
```

### ReAct Round No.3 Analysis:

```yaml
analysis: Microsoft (1975) is the oldest company, Google (1998) is the newest, and Apple (1976) is in between. The difference in years between Microsoft and Google needs to be calculated to determine how much older Microsoft is compared to Google.
planned_actions:
  - action_type: calculate_age
    description: Calculate the age difference between Microsoft and Google based on their founding years.
extension_needed: True
milestone: Calculate exact year difference between oldest and newest companies.
action_strategy: concurrent
reason:
  title: Founding years identified for comparison
  content: Apple was founded in 1976, Google in 1998, and Microsoft in 1975. Microsoft is the oldest company (1975). Google is the newest (1998). Next, calculate how much older Microsoft is compared to Google.
  confidence_score: 0.98
action_required: True
action_requests:
  - function: calculate_age
    arguments:
      founded_year: 1975
  - function: calculate_age
    arguments:
      founded_year: 1998
action_responses:
  - function: calculate_age
    arguments:
      founded_year: 1975
    output: 50
  - function: calculate_age
    arguments:
      founded_year: 1998
    output: 27
```

### ReAct Round No.4 Analysis:

```yaml
analysis: Microsoft (50 years old) is the oldest company, Google (27 years old) is the newest, and Apple (48 years old) is between them. The difference between Microsoft and Google is 23 years, thus Microsoft is 23 years older than Google.
planned_actions:
extension_needed: False
milestone: Final comparison and age difference calculated.
action_strategy: concurrent
reason:
  title: Calculate difference in founding years to determine age gap
  content: Microsoft was founded 50 years ago (in 1975) and Google 27 years ago (in 1998). To find how much older Microsoft is compared to Google, subtract 27 from 50, resulting in 23 years. Hence, Microsoft is 23 years older than Google.
  confidence_score: 0.99
action_required: False
action_requests:
action_responses:
```

### ReAct Final Answer:

```yaml
answer: Microsoft is the oldest company, founded in 1975. Google is the newest, founded in 1998. Microsoft is 23 years older than Google.
```


📊 Research Result:
Microsoft is the oldest company, founded in 1975. Google is the newest, founded in 1998. Microsoft is 23 years older than Google.


## ReAct vs Regular Function Calling

Let's compare ReAct with regular `operate()` to see the difference:

In [7]:
# Regular function calling approach
regular_branch = Branch(
    system="Answer questions about companies.",
    tools=[lookup_company, calculate_age],
    chat_model=iModel(model="openai/gpt-4.1-mini"),
)

# Same question, different approach
regular_result = await regular_branch.operate(
    instruction=question, actions=True
)

print("🔧 Regular operate() result:")
print(f"\nTools used: {len(regular_result.action_responses)}")

print("\n" + "=" * 50)
print(
    "🧠 ReAct approach showed more reasoning steps and systematic problem solving!"
)

🔧 Regular operate() result:

Tools used: 3

🧠 ReAct approach showed more reasoning steps and systematic problem solving!


## When to Use ReAct

**ReAct is best for:**
- Complex, multi-step problems
- When you want to see the reasoning process
- Problems that need strategic tool use
- Research and analysis tasks

**Regular `operate()` is better for:**
- Simple, single-step actions
- When you just want the answer quickly
- Straightforward tool usage

## Simple ReAct Configuration

Key parameters you can control:

In [8]:
# Different ReAct configurations
demo_branch = Branch(
    system="You solve problems step by step.",
    tools=[add, multiply, divide],
    chat_model=iModel(model="openai/gpt-4.1-mini"),
)

# Quick ReAct (fewer steps)
quick_result = await demo_branch.ReAct(
    instruct={"instruction": "What's 10 * 5 + 3?"},
    max_extensions=10,
    verbose=False,  # Hide reasoning steps
)

print("Quick ReAct:", quick_result)

# Detailed ReAct (more steps)
detailed_result = await demo_branch.ReAct(
    instruct={"instruction": "Calculate (15 * 4) + (20 / 5) - 3"},
    max_extensions=10,
    verbose=True,  # Show reasoning steps
)

print("\nDetailed ReAct:", detailed_result)

Quick ReAct: 53


### ReAct Round No.1 Analysis:

```yaml
analysis: The expression involves multiplication, division, addition, and subtraction. According to order of operations, multiplication and division go first, followed by addition and subtraction.
planned_actions:
  - action_type: multiply
    description: Calculate 15 multiplied by 4.
  - action_type: divide
    description: Calculate 20 divided by 5.
extension_needed: True
milestone: Obtain intermediate results of multiplication and division
action_strategy: concurrent
reason:
  title: Break down the expression step-by-step
  content: To calculate (15 * 4) + (20 / 5) - 3, first multiply 15 by 4, then divide 20 by 5, then add the two results, and finally subtract 3.
  confidence_score: 1.0
```

### ReAct Round No.2 Analysis:

```yaml
analysis: The first step applies multiplication and division first as per order of operation: 15 * 4 and 20 / 5. Next step will use the results to continue.
planned_actions:
  - action_type: multiply
    description: Calculate 15 times 4.
  - action_type: divide
    description: Calculate 20 divided by 5.
extension_needed: True
milestone: Obtain results of multiplication and division
action_strategy: concurrent
reason:
  title: Execute multiplication and division simultaneously
  content: Calculate 15 multiplied by 4 and 20 divided by 5 concurrently to obtain the intermediate results for addition and subtraction in the expression.
  confidence_score: 1.0
action_required: True
action_requests:
  - function: multiply
    arguments:
      a: 15
      b: 4
  - function: divide
    arguments:
      a: 20
      b: 5
action_responses:
  - function: multiply
    arguments:
      a: 15
      b: 4
    output: 60
  - function: divide
    arguments:
      a: 20
      b: 5
    output: 4.0
```

### ReAct Round No.3 Analysis:

```yaml
analysis: First adding the multiplication and division results (60 + 4.0) before subtracting 3 in the next step.
planned_actions:
  - action_type: add
    description: Add 60 and 4.0 as intermediate step before subtraction.
extension_needed: True
milestone: Obtain sum of multiplication and division results
action_strategy: sequential
reason:
  title: Add intermediate results and subtract 3
  content: We have the results from previous steps: 15 * 4 = 60 and 20 / 5 = 4.0. Now sum these results and then subtract 3 from the total.
  confidence_score: 1.0
action_required: True
action_requests:
  - function: add
    arguments:
      a: 60
      b: 4.0
action_responses:
  - function: add
    arguments:
      a: 60
      b: 4.0
    output: 64.0
```

### ReAct Round No.4 Analysis:

```yaml
analysis: Using addition with negative number to perform subtraction as the final step for the expression evaluation.
planned_actions:
  - action_type: add
    description: Subtract 3 from 64.0 by adding -3.
extension_needed: True
milestone: Obtain final calculation result after subtraction
action_strategy: sequential
reason:
  title: Subtract 3 from the sum
  content: We obtained 64.0 by adding 60 (15 * 4) and 4.0 (20 / 5). The last step is to subtract 3 from 64.0 to complete the expression (15 * 4) + (20 / 5) - 3.
  confidence_score: 1.0
action_required: True
action_requests:
  - function: add
    arguments:
      a: 64.0
      b: -3
action_responses:
  - function: add
    arguments:
      a: 64.0
      b: -3
    output: 61.0
```

### ReAct Round No.5 Analysis:

```yaml
analysis: The expression was evaluated by first performing multiplication and division: 15*4=60 and 20/5=4. Adding these gives 64, then subtracting 3 yields the final result 61.0.
planned_actions:
  - action_type: None
    description: No further calculations needed; final answer obtained.
extension_needed: False
milestone: Final answer obtained for the expression (15 * 4) + (20 / 5) - 3
action_strategy: sequential
reason:
  title: Final Step: Confirm final answer
  content: After completing the subtraction step to get 61.0, the entire expression (15 * 4) + (20 / 5) - 3 has been fully evaluated. The final answer is 61.0.
  confidence_score: 1.0
action_required: False
action_requests: None
action_responses:
  - function: add
    arguments:
      a: 64.0
      b: -3
    output: 61.0
```

### ReAct Final Answer:

```yaml
answer: 61.0
```


Detailed ReAct: 61.0


## Key Takeaways

**ReAct Pattern:**
1. 🧠 **Think** about what needs to be done
2. 🔧 **Act** using available tools
3. 🔄 **Repeat** until problem is solved

**Benefits:**
- More reliable problem solving
- Transparent reasoning process
- Better handling of complex tasks
- Strategic tool usage

**Next**: You now understand ReAct fundamentals! This prepares you for more advanced reasoning patterns and complex workflows.